In [1]:
import pandas as pd
import os
import torch 
import clip

/Data/ComputerVision_Project/env_recipe/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
summary_bert_path = "/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Summaries/export_summary_Bert.csv"
summary_bert = pd.read_csv(summary_bert_path)

data_dir = "ComputerVision_Data"
images_dir = "Food Images/Food Images"

In [3]:
recipe = summary_bert["Summary"]
print("number of sumary recipe ",len(recipe))

images = os.listdir(os.path.join(data_dir, images_dir))
print("number of images ",len(images))

number of sumary recipe  13463
number of images  3486


In [4]:
model_name = 'RN50x4'
model, preprocess = clip.load(model_name)
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

100%|███████████████████████████████████████| 402M/402M [02:36<00:00, 2.70MiB/s]
/Data/ComputerVision_Project/env_recipe/lib64/python3.9/site-packages/torch/cuda/__init__.py:104: UserWarning: 
NVIDIA RTX A2000 12GB with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70 sm_75.
If you want to use the NVIDIA RTX A2000 12GB GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [21]:
var=0

# Creation of a tensor for the text 
device = "cuda" if torch.cuda.is_available() else "cpu"
text_tensor = torch.zeros(len(summary_bert), context_length, dtype=torch.long)
labels = []
for i, row in enumerate(summary_bert.iterrows()):
    summary = row[1]["Summary"]
    if i==0: 
        print(summary)
    if len(summary.split()) > context_length:
        var+=1
        summary = summary[:context_length]  # Truncate the text if it's too long
    text_tensor[i] = clip.tokenize([summary]).to(device)  # Access row data using integer indices
    labels.append(row[1]["Image_Name"])  # Access row data using integer indices

print(var)



Roast chicken in a large cast-iron skillet until an instant-read thermometer inserted into the thickest part of breast registers 155°F, 50–60 minutes. Meanwhile, roast squash on lower rack until mostly tender, about 25 minutes.


RuntimeError: Input Place 2 chamomile tea bags in a heatsafe vessel, such as a large liquid measuring cup. Pour in 1 ½ cups boiling water, and let steep 5 minutes, then remove tea bags.
Add 1 ½ oz. reposado tequila, ¾ oz. fresh lemon juice, and 1 Tbsp. agave nectar and stir until incorporated. Pour into a 16-ounce insulated mug (or two smaller 8-ounce mugs) and drink hot. is too long for context length 77

In [23]:
test = "Input Place 2 chamomile tea bags in a heatsafe vessel, such as a large liquid measuring cup. Pour in 1 ½ cups boiling water, and let steep 5 minutes, then remove tea bags. Add 1 ½ oz. reposado tequila, ¾ oz. fresh lemon juice, and 1 Tbsp. agave nectar and stir until incorporated. Pour into a 16-ounce insulated mug (or two smaller 8-ounce mugs) and drink hot."

In [27]:
new = test.split()[:30]
new = "".join(new)
clip.tokenize([new]).to(device)

tensor([[49406,   512, 29535,  1445,   273,  1290,   547, 18398,   600,   596,
           602,  1303,    64, 12175,  2901, 16387,   267,   605, 10717, 34333,
            70,  1018,   666,  1568,  5867,   565,   653,  1937,   269, 33112,
           530,   272, 33613,   729,  3982,   647,   543,   653,  1573,   267,
           672,  1898, 20714,   276,  3056,   267,  1594, 49407,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]], device='cuda:0')

In [ ]:
torch.save(text_tensor, "text_tensor.pt")